In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from skimage import transform
from sklearn.model_selection import train_test_split



In [ ]:
torch.cuda.is_available()

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

In [ ]:

    
def to_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def read_im(folder):
    imlist = []
    for file in os.listdir("data/" + folder):   #https://stackoverflow.com/questions/72022176/warning-cant-open-read-file-check-file-path-integrity
        im = plt.imread("data/" + folder + "/"+ file)
        im = im[:200,50:250]
        im = transform.resize(im, (126, 126), mode='reflect', anti_aliasing=True)  
        im= to_gray(im)
        #im = im.flatten()
        shape = im.shape
        imlist.append(im.reshape(1,shape[0], shape[1]))
        
    images = np.concatenate(imlist, axis =0)
    print(f'Reading {folder}: {images.shape}')
    return images
#generate data labes for images in folders
def generate_lables(image_data):
   labels = [np.repeat(i,image_data[i].shape[0]) for i in range(len(image_data))]
   return np.concatenate(labels, axis=0)

In [ ]:
names = ["rock","paper", "scissors"]
im_data = [read_im(folder) for folder in names]
labels = generate_lables(im_data)


In [ ]:
print(im_data[0][0].shape)
plt.imshow(im_data[0][0])
data = np.concatenate(im_data, axis =0)

In [ ]:
labels.shape

In [ ]:
original_shape = data.shape

In [ ]:

data = data.reshape(-1,1,126,126)

data.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
batch_size = 8
X_train = torch.from_numpy(X_train.reshape(-1,1,126,126))
y_train = torch.from_numpy(y_train)
X_test = torch.from_numpy(X_test.reshape(-1,1,126,126))
y_test = torch.from_numpy(y_test)

classes = ("rock", "paper", "scissors")

X_train.shape

In [ ]:
from torch.utils.data import TensorDataset, DataLoader


# Create dataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
print(f'Number of batches in the training set: {int(len(train_dataset) / batch_size)}')

In [ ]:
# trainloader = torch.utils.data.DataLoader(trainset, batch_size= batch_size,
#                                           shuffle = True, num_workers=2)
# valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
#                                         shuffle= False, num_workers=2)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)

In [ ]:
import torch.nn as nn 
import torch.nn.functional as F 

class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.Cov1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3)
        self.pool1 = nn.MaxPool2d(2,2)
        
        self.Cov2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.Cov3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3)
        self.pool3 = nn.MaxPool2d(2,2)
        
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(25088, 512)
        self.drop1 = nn.Dropout(p=0.3)
        
        self.fc2 = nn.Linear(in_features=512, out_features=512)
        self.drop2 = nn.Dropout(p=0.3)
        
        self.out = nn.Linear(in_features=512, out_features=3)
        

    def forward(self,x):
        x = F.relu(self.Cov1(x))
        x = self.pool1(x)
        
        x = F.relu(self.Cov2(x))
        x = self.pool2(x)
        
        x = F.relu(self.Cov3(x))
        x = self.pool3(x)
        
        x = self.flatten(x)
        
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        
        x = F.relu(self.fc2(x))
        x = self.drop2(x)
        
        x = self.out(x)
        
        return x

In [ ]:
net = NeuralNet()
net.to(device)

In [ ]:
for i, data in enumerate(train_loader):
    inputs, labels = data[0].to(device), data[1].to(device)
    print(f'Input shape: {inputs.shape}')
    print(f'After network shape: {net(inputs).shape}')
    
    break

In [ ]:
num_params = 0
for x in net.parameters():
    num_params += len(torch.flatten(x))
    
print(f'Number of parameters in the model: {num_params:,}')

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.0001)

In [ ]:
loss_lst = []
acc_lst = []
def train_one_epoch():
    net.train(True)
    
    running_loss = 0.0
    running_accuracy = 0.0
    
    for batch_index, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device) 
        
        optimizer.zero_grad()
        
        outputs = net(inputs) # shape: [batch_size, 10]
        correct = torch.sum(labels == torch.argmax(outputs, dim=1)).item()
        running_accuracy += correct / batch_size
        
        loss = criterion(outputs,labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if batch_index % 100 == 99:
            avg_loss_across_batches = running_loss / 100
            avg_acc_across_batches = (running_accuracy / 100) * 100
            print('Batch {0}, loss: {1:.3f}, Accuracy: {2:.1f}%'.format(batch_index+1, avg_loss_across_batches,
                                                                        avg_acc_across_batches))
            loss_lst.append(avg_loss_across_batches)
            acc_lst.append(avg_acc_across_batches)
            running_loss = 0.0
            running_accuracy = 0.0
            
    print()

In [ ]:
def validate_one_epoch():
    net.train(False)
    running_loss = 0.0
    running_accuracy = 0.0
    
    for i, data in enumerate(test_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        with torch.no_grad():
            outputs = net(inputs)
            correct = torch.sum(labels == torch.argmax(outputs, dim=1)).item()
            running_accuracy += correct / batch_size
            loss = criterion(outputs,labels)
            running_loss += loss.item()
                    
    avg_loss_across_batches = running_loss / len(test_loader)
    avg_acc_across_batches = (running_accuracy / len(test_loader)) * 100
    
    print('Val loss: {0: .3f}, Val Accuracy: {1: .1f}%'.format(avg_loss_across_batches,
                                                            avg_acc_across_batches))
    
    print('************************************************************')
    print()

In [ ]:
num_epochs = 10

for epoch_index in range(num_epochs):
    print(f'Epoch: {epoch_index + 1}\n')
    
    train_one_epoch()
    validate_one_epoch()
    
    print('Finished Training')

In [ ]:
plt.plot(loss_lst)

In [ ]:
plt.plot(acc_lst)

In [ ]:
trained_CNN = torch.save(net, 'model.pth')

In [ ]:
# import random

# def rotate_image(image, angle):
#   """Rotates an image by a given angle."""
#   center = tuple(np.array(image.shape[1::-1]) / 2)
#   rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
#   result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
#   return result

# def rotate_dataset(data):
#   data = data
#   for i in range(len(data)):
#     # Generate a random angle between 0 and 360 degrees
#     angle = random.randint(0, 360)
#     # Rotate the image
#     data[i] = rotate_image(data[i], angle)
#     # Save the rotated image
#     cv2.imwrite(f'processed/rotated_image{i}.jpg', data[i])
#   return data
    
    

In [ ]:
def random_rotate_tensor(tensor_dataset):
    """
    Rotates each image in the tensor dataset randomly by up to 360 degrees.
    Preserves the original shape of the dataset.
    
    Args:
        tensor_dataset (torch.Tensor): Input tensor of shape (N, C, H, W)
    
    Returns:
        torch.Tensor: Rotated tensor of the same shape as input
        
    """
    if isinstance(tensor_dataset, list):
        tensor_dataset = torch.tensor(tensor_dataset, dtype=torch.float32)

    if not isinstance(tensor_dataset, torch.Tensor):
        raise TypeError("Input must be a PyTorch tensor or a list convertible to a tensor.")

    N, C, H, W = tensor_dataset.shape
    transform = transforms.RandomRotation(degrees=(0, 360))
    
    rotated_images = torch.zeros_like(tensor_dataset)
    
    for i in range(N):
        rotated_images[i] = transform(tensor_dataset[i])
    
    return rotated_images
rotated_data= random_rotate_tensor(data)
rotated_data.shape